In [ ]:
!jupyter kernelspec list

Available kernels:
  ir         /usr/local/share/jupyter/kernels/ir
  python3    /usr/local/share/jupyter/kernels/python3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Nov  5 02:52:34 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python3 -m pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchtext==0.15.1 torchaudio==2.0.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118
!python3 -m pip install torch-optimizer
!python3 -m pip install pyro-ppl
!python3 -m pip install madgrad

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 464.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.1/153.1 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for lit: filename=lit-17.0.4-py3-none-any.whl size=93257 sha256=2d6ffa12216e435b4609bf68fc890fc360cdde3e6f00131d099365e56fb4d56f
  Stored in directory: /root/.cach

In [ ]:

!gdown https://drive.google.com/uc?id="XXXXXXXXXXXXXXXXX"

Downloading...
From: https://drive.google.com/uc?id=1PBXoHUA808zny2PBuUXXC-m2tnQiDDUB
To: /content/library.tar.gz
100% 174M/174M [00:04<00:00, 36.1MB/s]


In [ ]:

%%shell

tar xvf library.tar.gz

library/abind/
library/abind/DESCRIPTION
library/abind/help/
library/abind/help/abind.rdb
library/abind/help/abind.rdx
library/abind/help/aliases.rds
library/abind/help/AnIndex
library/abind/help/paths.rds
library/abind/html/
library/abind/html/00Index.html
library/abind/html/R.css
library/abind/INDEX
library/abind/Meta/
library/abind/Meta/features.rds
library/abind/Meta/hsearch.rds
library/abind/Meta/links.rds
library/abind/Meta/nsInfo.rds
library/abind/Meta/package.rds
library/abind/Meta/Rd.rds
library/abind/NAMESPACE
library/abind/R/
library/abind/R/abind
library/abind/R/abind.rdb
library/abind/R/abind.rdx
library/abind/sccversion.txt
library/abind/svnversion.txt
library/bayesm/
library/bayesm/data/
library/bayesm/data/bank.rda
library/bayesm/data/camera.RData
library/bayesm/data/cheese.rda
library/bayesm/data/customerSat.rda
library/bayesm/data/detailing.rda
library/bayesm/data/margarine.rda
library/bayesm/data/orangeJuice.rda
library/bayesm/data/Scotch.rda
library/bayesm/data/tuna

In [ ]:
%%script R --save --restore
.libPaths("/content/library")

#devtools::install_github("https://github.com/TheoreticalEcology/s-jSDM/tree/v1.0.1", subdir = "sjSDM", dep=FALSE)

##read packages
lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")



save.dir <- "Output"

dir.create(save.dir)
#############3
jsdm_files <- readRDS("Input/jsdm_files_sPC90per.rds")

Occ <- jsdm_files[["Occ"]]
Env <- jsdm_files[["Env"]]
SP <- jsdm_files[["SP"]]


Spatial = generateSpatialEV(SP)


iter <- 100L
sampling <- 5000L
model = sjSDM_cv(Y = Occ,
              env = linear(Env,~.),
              spatial = linear(Spatial, ~0+.),
              tune="random",
              CV=5L ,
              tune_steps = 20L,
              biotic = bioticStruct(),
              se = F, family=binomial("probit"),
              device = "gpu",
              sampling=sampling,
              iter = iter) # increase iter for your own data



saveRDS(model,sprintf("%s/jsdm_sdist_tuning_iter%s_Msamp%s_sPC90per.rds",save.dir,iter,sampling))

model2<- sjSDM.tune(model)

saveRDS(model2,sprintf("%s/jsdm_sdist_result_iter%s_Msamp%s_sPC90per.rds",save.dir,iter,sampling))

In [ ]:
%%script R --save --restore
.libPaths("/content/library")
##read packages


lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)
lib <- "tidyr";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")


save.dir <- "Output"

dir.create(save.dir)

######

object <- readRDS("Output/jsdm_sdist_result_iter100_Msamp5000_sPC90per.rds")

pred <- predict(object)

res <- cbind(gather(as.data.frame(object$data$Y),OTU1,pres),gather(as.data.frame(pred),OTU2,prob))

saveRDS(res,sprintf("%s/pred_rawdata.rds",save.dir))

###
cov <- getCov(object)
saveRDS(cov,sprintf("%s/OTU_cov_97.rds",save.dir))



In [ ]:
%%script R --save --restore
.libPaths("/content/library")
##read packages

lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")


save.dir <- "Output"

dir.create(save.dir)

######

object <- readRDS("Output/jsdm_sdist_result_iter100_Msamp5000_sPC90per.rds")


#########
individual=TRUE
full_m = logLik(object, individual=individual)[[1]]


m1 = update(object, env_formula = ~plant, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A1BS_m = logLik(m1, individual=individual)[[1]]

m2 = update(object, env_formula = ~.-plant, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A2BS_m = logLik(m2, individual=individual)[[1]]

m3 = update(object, env_formula = NULL, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A1A2S_m = logLik(m3, individual=individual)[[1]]

m4 = update(object, env_formula = NULL, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A1A2B_m = logLik(m4, individual=individual)[[1]]

###

m5 = update(object, env_formula = ~plant, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A1B_m = logLik(m5, individual=individual)[[1]]

m6 = update(object, env_formula = ~.-plant, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A2B_m = logLik(m6, individual=individual)[[1]]

m7 = update(object, env_formula = ~plant, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A1S_m = logLik(m7, individual=individual)[[1]]

m8 = update(object, env_formula = ~.-plant,spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A2S_m = logLik(m8, individual=individual)[[1]]

m9 = update(object, env_formula = ~1, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
BS_m = logLik(m9, individual=individual)[[1]]

m10 = update(object, env_formula = NULL, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A1A2_m = logLik(m10, individual=individual)[[1]]

#######
m11 = update(object, env_formula = ~plant, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A1_m = logLik(m11, individual=individual)[[1]]

m12 = update(object, env_formula = ~.-plant, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A2_m = logLik(m12, individual=individual)[[1]]

m13 = update(object, env_formula = ~1, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
S_m = logLik(m13, individual=individual)[[1]]

m14 = update(object, env_formula = ~1, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
B_m = logLik(m14, individual=individual)[[1]]

###
m15 = update(object, env_formula = ~1, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
null_m = logLik(m15, individual=individual)[[1]]

#########
saveRDS(list(NoSoil=m1,NoPlant=m2,NoBio=m3,NoSpatial=m4,PlBi=m5,SoBi=m6,PlSp=m7,SoSp=m8,BiSp=m9,PlSo=m10,Pl=m11,So=m12,Bi=m13,Sp=m14,Null=m15),sprintf("%s/restricted_models_sPC90per.rds",save.dir))
saveRDS(list(Full=full_m,NoSoil=A1BS_m,NoPlant=A2BS_m,NoBio=A1A2S_m,NoSpatial=A1A2B_m,PlBi=A1B_m,SoBi=A2B_m,PlSp=A1S_m,SoSp=A2S_m,PlSo=A1A2_m,BiSp=BS_m,Pl=A1_m,So=A2_m,Bi=B_m,Sp=S_m,Null=null_m),sprintf("%s/restricted_lls_mat_sPC90per.rds",save.dir))


OTU sequence similarity threshold 93%

In [ ]:
%%script R --save --restore
.libPaths("/content/library")

#devtools::install_github("https://github.com/TheoreticalEcology/s-jSDM/tree/v1.0.1", subdir = "sjSDM", dep=FALSE)

##read packages
lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")



save.dir <- "Output"

dir.create(save.dir)
#############3
jsdm_files <- readRDS("Input/jsdm_files_sPC90per_OTU93.rds")

Occ <- jsdm_files[["Occ"]]
Env <- jsdm_files[["Env"]]
SP <- jsdm_files[["SP"]]


Spatial = generateSpatialEV(SP)


iter <- 100L
sampling <- 5000L
model = sjSDM_cv(Y = Occ,
              env = linear(Env,~.),
              spatial = linear(Spatial, ~0+.),
              tune="random",
              CV=5L ,
              tune_steps = 20L,
              biotic = bioticStruct(),
              se = F, family=binomial("probit"),
              device = "gpu",
              sampling=sampling,
              iter = iter) # increase iter for your own data



saveRDS(model,sprintf("%s/jsdm_sdist_tuning_iter%s_Msamp%s_OTU93.rds",save.dir,iter,sampling))

model2<- sjSDM.tune(model)

saveRDS(model2,sprintf("%s/jsdm_sdist_result_iter%s_Msamp%s_OTU93.rds",save.dir,iter,sampling))


R version 4.3.1 (2023-06-16) -- "Beagle Scouts"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> .libPaths("library")
> 
> devtools::install_github("https://github.com/TheoreticalEcology/s-jSDM/tree/v1.0.1", subdir = "sjSDM", dep=FALSE)
> 
> ##read packages
> lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)
[1] ‘1.0.1’
> 
> ##set directory
> setwd("/content/drive/MyDrive/go

Skipping install of 'sjSDM' from a github remote, the SHA1 (cf0bc0a0) has not changed since last install.
  Use `force = TRUE` to force installation
── Attaching sjSDM ──────────────────────────────────────────────────── 1.0.1 ──
✔ torch <environment> 
✔ torch_optimizer  
✔ pyro  
✔ madgrad  

Warning message:
In dir.create(save.dir) : '231104_sjSDM_soildist_OTU93' already exists
/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:200: UserWarning: operator() profile_node %444 : int[] = prim::profile_ivalue(%dims.12)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [ ]:
%%script R --save --restore
.libPaths("/content/library")
##read packages


lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)
lib <- "tidyr";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")


save.dir <- "Output"

dir.create(save.dir)

######

object <- readRDS("Output/jsdm_sdist_result_iter100_Msamp5000_OTU93.rds")

pred <- predict(object)

res <- cbind(gather(as.data.frame(object$data$Y),OTU1,pres),gather(as.data.frame(pred),OTU2,prob))

saveRDS(res,sprintf("%s/pred_rawdata_OTU93.rds",save.dir))

###
cov <- getCov(object)
saveRDS(cov,sprintf("%s/OTU_cov_93.rds",save.dir))




R version 4.3.1 (2023-06-16) -- "Beagle Scouts"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> .libPaths("library")
> ##read packages
> 
> devtools::install_github("https://github.com/TheoreticalEcology/s-jSDM/tree/v1.0.1", subdir = "sjSDM", dep=FALSE)
── R CMD build ─────────────────────────────────────────────────────────────────
* checking for file ‘/tmp/Rtmp7Rgbpr/remotes69569b8749a/TheoreticalEco

Installing package into ‘/content/library’
(as ‘lib’ is unspecified)
* installing *source* package ‘sjSDM’ ...
** using staged installation
** R
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
*** copying figures
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (sjSDM)
── Attaching sjSDM ──────────────────────────────────────────────────── 1.0.1 ──
✔ torch <environment> 
✔ torch_optimizer  
✔ pyro  
✔ madgrad  

Warning message:
In dir.create(save.dir) : '231104_sjSDM_soildist_OTU93' already exists


In [ ]:
%%script R --save --restore
.libPaths("/content/library")
##read packages

lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")


save.dir <- "Output"

dir.create(save.dir)

######

object <- readRDS("Output/jsdm_sdist_result_iter100_Msamp5000_OTU93.rds")


#########
individual=TRUE
full_m = logLik(object, individual=individual)[[1]]


m1 = update(object, env_formula = ~plant, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A1BS_m = logLik(m1, individual=individual)[[1]]

m2 = update(object, env_formula = ~.-plant, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A2BS_m = logLik(m2, individual=individual)[[1]]

m3 = update(object, env_formula = NULL, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A1A2S_m = logLik(m3, individual=individual)[[1]]

m4 = update(object, env_formula = NULL, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A1A2B_m = logLik(m4, individual=individual)[[1]]

###

m5 = update(object, env_formula = ~plant, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A1B_m = logLik(m5, individual=individual)[[1]]

m6 = update(object, env_formula = ~.-plant, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A2B_m = logLik(m6, individual=individual)[[1]]

m7 = update(object, env_formula = ~plant, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A1S_m = logLik(m7, individual=individual)[[1]]

m8 = update(object, env_formula = ~.-plant,spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A2S_m = logLik(m8, individual=individual)[[1]]

m9 = update(object, env_formula = ~1, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
BS_m = logLik(m9, individual=individual)[[1]]

m10 = update(object, env_formula = NULL, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A1A2_m = logLik(m10, individual=individual)[[1]]

#######
m11 = update(object, env_formula = ~plant, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A1_m = logLik(m11, individual=individual)[[1]]

m12 = update(object, env_formula = ~.-plant, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A2_m = logLik(m12, individual=individual)[[1]]

m13 = update(object, env_formula = ~1, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
S_m = logLik(m13, individual=individual)[[1]]

m14 = update(object, env_formula = ~1, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
B_m = logLik(m14, individual=individual)[[1]]

###
m15 = update(object, env_formula = ~1, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
null_m = logLik(m15, individual=individual)[[1]]

#########
saveRDS(list(NoSoil=m1,NoPlant=m2,NoBio=m3,NoSpatial=m4,PlBi=m5,SoBi=m6,PlSp=m7,SoSp=m8,BiSp=m9,PlSo=m10,Pl=m11,So=m12,Bi=m13,Sp=m14,Null=m15),sprintf("%s/restricted_models_OTU93.rds",save.dir))
saveRDS(list(Full=full_m,NoSoil=A1BS_m,NoPlant=A2BS_m,NoBio=A1A2S_m,NoSpatial=A1A2B_m,PlBi=A1B_m,SoBi=A2B_m,PlSp=A1S_m,SoSp=A2S_m,PlSo=A1A2_m,BiSp=BS_m,Pl=A1_m,So=A2_m,Bi=B_m,Sp=S_m,Null=null_m),sprintf("%s/restricted_lls_mat_OTU93.rds",save.dir))



R version 4.3.1 (2023-06-16) -- "Beagle Scouts"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> .libPaths("library")
> ##read packages
> 
> devtools::install_github("https://github.com/TheoreticalEcology/s-jSDM/tree/v1.0.1", subdir = "sjSDM", dep=FALSE)
> 
> lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)
[1] ‘1.0.1’
> 
> ##set directory
> setwd("/content/drive/MyDrive/go

Skipping install of 'sjSDM' from a github remote, the SHA1 (cf0bc0a0) has not changed since last install.
  Use `force = TRUE` to force installation
── Attaching sjSDM ──────────────────────────────────────────────────── 1.0.1 ──
✔ torch <environment> 
✔ torch_optimizer  
✔ pyro  
✔ madgrad  

Warning message:
In dir.create(save.dir) : '231104_sjSDM_soildist_OTU93' already exists
/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


ph

In [ ]:
%%script R --save --restore
.libPaths("/content/library")

##read packages
lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")


save.dir <- "Output"

dir.create(save.dir)

#############3
jsdm_files <- readRDS("Input/jsdm_files_ph.rds")

Occ <- jsdm_files[["Occ"]]
Env <- jsdm_files[["Env"]]
SP <- jsdm_files[["SP"]]


Spatial = generateSpatialEV(SP)


iter <- 100L
sampling <- 5000L
model = sjSDM_cv(Y = Occ,
              env = linear(Env,~.),
              spatial = linear(Spatial, ~0+.),
              tune="random",
              CV=5L ,
              tune_steps = 20L,
              biotic = bioticStruct(),
              se = F, family=binomial("probit"),
              device = "gpu",
              sampling=sampling,
              iter = iter) # increase iter for your own data



saveRDS(model,sprintf("%s/jsdm_sd_ph_tuning_iter%s_Msamp%s.rds",save.dir,iter,sampling))

model2<- sjSDM.tune(model)

saveRDS(model2,sprintf("%s/jsdm_sd_ph_result_iter%s_Msamp%s.rds",save.dir,iter,sampling))


R version 4.3.0 (2023-04-21) -- "Already Tomorrow"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> .libPaths("library")
> 
> devtools::install_github("https://github.com/TheoreticalEcology/s-jSDM/tree/v1.0.1", subdir = "sjSDM", dep=FALSE)
> 
> ##read packages
> lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)
[1] ‘1.0.1’
> 
> ##set directory
> setwd("/content/drive/MyDrive

Skipping install of 'sjSDM' from a github remote, the SHA1 (cf0bc0a0) has not changed since last install.
  Use `force = TRUE` to force installation
── Attaching sjSDM ──────────────────────────────────────────────────── 1.0.1 ──
✔ torch <environment> 
✔ torch_optimizer  
✔ pyro  
✔ madgrad  

Warning message:
In dir.create(save.dir) : '230527_sjSDM_soildist_ph_PC90' already exists
/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py:200: UserWarning: operator() profile_node %444 : int[] = prim::profile_ivalue(%dims.12)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


In [ ]:
%%script R --save --restore
.libPaths("/content/library")
##read packages


lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)
lib <- "tidyr";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")


save.dir <- "Output"

dir.create(save.dir)

######

object <- readRDS("Output/jsdm_sd_ph_result_iter100_Msamp5000.rds")

pred <- predict(object)

res <- cbind(gather(as.data.frame(object$data$Y),OTU1,pres),gather(as.data.frame(pred),OTU2,prob))

saveRDS(res,sprintf("%s/pred_rawdata_ph.rds",save.dir))

###
cov <- getCov(object)
saveRDS(cov,sprintf("%s/OTU_cov_ph.rds",save.dir))


In [ ]:
%%script R --save --restore
.libPaths("/content/library")

##read packages
lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)

##set directory
setwd("/content/drive/MyDrive/google_Colab")


save.dir <- "Output"

dir.create(save.dir)

######

object <- readRDS("Output/jsdm_sd_ph_result_iter100_Msamp5000.rds")


#########
models <- list(NULL)
lls <- list(NULL)

individual=TRUE
full_m = logLik(object, individual=individual)[[1]]
models[[1]] <- object
lls[[1]] <- full_m

m1 = update(object, env_formula = ~.-ph, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A1A2BS_m = logLik(m1, individual=individual)[[1]]
models[[2]] <- m1
lls[[2]] <- A1A2BS_m

m2 = update(object, env_formula = ~plant+ph, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A1A3BS_m = logLik(m2, individual=individual)[[1]]
models[[3]] <- m2
lls[[3]] <- A1A3BS_m

m3 = update(object, env_formula = ~.-plant, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A2A3BS_m = logLik(m3, individual=individual)[[1]]
models[[4]] <- m3
lls[[4]] <- A2A3BS_m

m4 = update(object, env_formula = NULL, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A1A2A3B_m = logLik(m4, individual=individual)[[1]]
models[[5]] <- m4
lls[[5]] <- A1A2A3B_m

m5 = update(object, env_formula = NULL, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A1A2A3S_m = logLik(m5, individual=individual)[[1]]
models[[6]] <- m5
lls[[6]] <- A1A2A3S_m
###
nam <- c("Full","Noph","NoSoil","NoPlant","NoSpatial","NoBio")
##
m6 = update(object, env_formula = ~.-ph, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A1A2B_m = logLik(m6, individual=individual)[[1]]
models[[7]] <- m6
lls[[7]] <- A1A2B_m

m7 = update(object, env_formula = ~.-plant, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A2A3B_m = logLik(m7, individual=individual)[[1]]
models[[8]] <- m7
lls[[8]] <- A2A3B_m

m8 = update(object, env_formula = ~plant+ph, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A1A3B_m = logLik(m8, individual=individual)[[1]]
models[[9]] <- m8
lls[[9]] <- A1A3B_m

m9 = update(object, env_formula = ~.-ph,spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A1A2S_m = logLik(m9, individual=individual)[[1]]
models[[10]] <- m9
lls[[10]] <- A1A2S_m

m10 = update(object, env_formula = ~plant+ph,spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A1A3S_m = logLik(m10, individual=individual)[[1]]
models[[11]] <- m10
lls[[11]] <- A1A3S_m

m11 = update(object, env_formula = ~.-plant,spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A2A3S_m = logLik(m11, individual=individual)[[1]]
models[[12]] <- m11
lls[[12]] <- A2A3S_m

m12 = update(object, env_formula = ~plant, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A1BS_m = logLik(m12, individual=individual)[[1]]
models[[13]] <- m12
lls[[13]] <- A1BS_m

m13 = update(object, env_formula = ~.-ph-plant, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A2BS_m = logLik(m13, individual=individual)[[1]]
models[[14]] <- m13
lls[[14]] <- A2BS_m

m14 = update(object, env_formula = ~ph, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
A3BS_m = logLik(m14, individual=individual)[[1]]
models[[15]] <- m14
lls[[15]] <- A3BS_m

m15 = update(object, env_formula = NULL, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A1A2A3_m = logLik(m15, individual=individual)[[1]]
models[[16]] <- m15
lls[[16]] <- A1A2A3_m
#######
nam2 <- c(nam,"PlSoBi","SophBi","PlphBi","PlSoSp","PlphSp","SophSp","PlBiSp","SoBiSp","phBiSp","PlSoph")
##

m16 = update(object, env_formula = ~.-ph, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A1A2_m = logLik(m16, individual=individual)[[1]]
models[[17]] <- m16
lls[[17]] <- A1A2_m

m17 = update(object, env_formula = ~plant+ph, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A1A3_m = logLik(m17, individual=individual)[[1]]
models[[18]] <- m17
lls[[18]] <- A1A3_m

m18 = update(object, env_formula = ~.-plant, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A2A3_m = logLik(m18, individual=individual)[[1]]
models[[19]] <- m18
lls[[19]] <- A2A3_m

m19 = update(object, env_formula = ~plant, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A1B_m = logLik(m19, individual=individual)[[1]]
models[[20]] <- m19
lls[[20]] <- A1B_m

m20 = update(object, env_formula = ~.-plant-ph, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A2B_m = logLik(m20, individual=individual)[[1]]
models[[21]] <- m20
lls[[21]] <- A2B_m

m21 = update(object, env_formula = ~ph, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
A3B_m = logLik(m21, individual=individual)[[1]]
models[[22]] <- m21
lls[[22]] <- A3B_m

m22 = update(object, env_formula = ~plant, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A1S_m = logLik(m22, individual=individual)[[1]]
models[[23]] <- m22
lls[[23]] <- A1S_m

m23 = update(object, env_formula = ~.-plant-ph, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A2S_m = logLik(m23, individual=individual)[[1]]
models[[24]] <- m23
lls[[24]] <- A2S_m

m24 = update(object, env_formula = ~ph, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
A3S_m = logLik(m24, individual=individual)[[1]]
models[[25]] <- m24
lls[[25]] <- A3S_m

m25 = update(object, env_formula = ~1, spatial_formula= NULL, biotic=bioticStruct(diag = FALSE ))
BS_m = logLik(m25, individual=individual)[[1]]
models[[26]] <- m25
lls[[26]] <- BS_m
##
nam3 <- c(nam2,"PlSo","Plph","Soph","PlBi","SoBi","phBi","PlSp","SoSp","phSp","BiSp")
#

m26 = update(object, env_formula = ~plant, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A1_m = logLik(m26, individual=individual)[[1]]
models[[27]] <- m26
lls[[27]] <- A1_m

m27 = update(object, env_formula = ~.-plant-ph, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A2_m = logLik(m27, individual=individual)[[1]]
models[[28]] <- m27
lls[[28]] <- A2_m

m28 = update(object, env_formula = ~ph, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
A3_m = logLik(m28, individual=individual)[[1]]
models[[29]] <- m28
lls[[29]] <- A3_m

m29 = update(object, env_formula = ~1, spatial_formula= NULL, biotic=bioticStruct(diag = TRUE ))
S_m = logLik(m29, individual=individual)[[1]]
models[[30]] <- m29
lls[[30]] <- S_m

m30 = update(object, env_formula = ~1, spatial_formula= ~0, biotic=bioticStruct(diag = FALSE ))
B_m = logLik(m30, individual=individual)[[1]]
models[[31]] <- m30
lls[[31]] <- B_m

###
m31 = update(object, env_formula = ~1, spatial_formula= ~0, biotic=bioticStruct(diag = TRUE ))
null_m = logLik(m31, individual=individual)[[1]]
models[[32]] <- m31
lls[[32]] <- null_m

##
nam4 <- c(nam3,"Pl","So","ph","Bi","Sp","Null")
#
names(models) <- nam4
names(lls) <- nam4

#########
#test
head(lls[[1]])

head(lls[[7]])

llr <- data.frame(pH=colSums(A1A2BS_m)-colSums(full_m),
Soil=colSums(A1A3BS_m)-colSums(full_m),
Plant=colSums(A2A3BS_m)-colSums(full_m),
Biotic=colSums(A1A2A3S_m)-colSums(full_m),
Spatial=colSums(A1A2A3B_m)-colSums(full_m)
)

write.csv(llr,sprintf("%s/llr_test.csv",save.dir))
####

saveRDS(models,sprintf("%s/restricted_models_ph.rds",save.dir))
saveRDS(lls,sprintf("%s/restricted_lls_mat_ph.rds",save.dir))



R version 4.3.0 (2023-04-21) -- "Already Tomorrow"
Copyright (C) 2023 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under certain conditions.
Type 'license()' or 'licence()' for distribution details.

  Natural language support but running in an English locale

R is a collaborative project with many contributors.
Type 'contributors()' for more information and
'citation()' on how to cite R or R packages in publications.

Type 'demo()' for some demos, 'help()' for on-line help, or
'help.start()' for an HTML browser interface to help.
Type 'q()' to quit R.

> .libPaths("library")
> 
> devtools::install_github("https://github.com/TheoreticalEcology/s-jSDM/tree/v1.0.1", subdir = "sjSDM", dep=FALSE)
> 
> ##read packages
> lib <- "sjSDM";library(package = lib, character.only=TRUE);packageVersion(lib)
[1] ‘1.0.1’
> 
> ##set directory
> setwd("/content/drive/MyDrive

Skipping install of 'sjSDM' from a github remote, the SHA1 (cf0bc0a0) has not changed since last install.
  Use `force = TRUE` to force installation
── Attaching sjSDM ──────────────────────────────────────────────────── 1.0.1 ──
✔ torch <environment> 
✔ torch_optimizer  
✔ pyro  
✔ madgrad  

Warning message:
In dir.create(save.dir) : '230527_sjSDM_soildist_ph_PC90' already exists
/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
